# 4.9 Task Part 1

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [3]:
# Import Data
df_customer = pd.read_csv(r'C:\Users\User\Documents\07-2022 Instacart Basket Analysis\02 Data\Original Data\customers.csv', index_col = False)

#### Wrangle the data

In [4]:
df_customer.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [5]:
# Rename Surnam column to Surname
df_customer.rename(columns = {'Surnam' : 'Surname'}, inplace = True)

In [7]:
# Lowercase STATE
df_customer.rename(columns = {'STATE' : 'State'}, inplace = True)

In [9]:
# Capitalize I
df_customer.rename(columns = {'income' : 'Income'}, inplace = True)

In [10]:
# Drop date_joined
df_customer.drop(columns = ['date_joined'])

,user_id,First Name,Surname,Gender,State,Age,n_dependants,fam_status,Income
0,26711,Deborah,Esquivel,Female,Missouri,48,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1,married,40374
...,...,...,...,...,...,...,...,...,...
206204,168073,Lisa,Case,Female,North Carolina,44,1,married,148828
206205,49635,Jeremy,Robbins,Male,Hawaii,62,3,married,168639
206206,135902,Doris,Richmond,Female,Missouri,66,2,married,53374
206207,81095,Rose,Rollins,Female,California,27,1,married,99799


### Conduct consistency checks

In [11]:
df_customer.describe()

,user_id,Age,n_dependants,Income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [12]:
# Initial glance appears logical

In [14]:
# Check for mixed-type columns
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].applymap(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

First Name


In [16]:
# Convert data type
df_customer['First Name'] = df_customer['First Name'].astype('str')

In [17]:
# Number of missing values
df_customer.isnull().sum()

user_id         0
First Name      0
Surname         0
Gender          0
State           0
Age             0
date_joined     0
n_dependants    0
fam_status      0
Income          0
dtype: int64

In [18]:
# No missing values

In [19]:
# Create df with duplicates
df_dups = df_customer[df_customer.duplicated()]

In [20]:
# View duplicates
df_dups

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,Income


In [21]:
# No duplicates

### Combine customer data with rest of Instacart data

In [22]:
# Import Instacart data
ords_prods_merge = pd.read_pickle(r"C:\Users\User\Documents\07-2022 Instacart Basket Analysis\02 Data\Prepared Data\orders_products_merged.pkl")

In [23]:
df_customer.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,Income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [24]:
ords_prods_merge.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,...,_merge,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,med_freq,freq_flag
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,...,both,Regularly busy,Busiest days,Fewest orders,32,Regular customer,6.935811,Low spender,8.0,frequent customer
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,...,both,Regularly busy,Busiest days,Fewest orders,32,Regular customer,6.935811,Low spender,8.0,frequent customer
2,0,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,...,both,Busiest day,Busiest days,Fewest orders,5,New customer,7.930208,Low spender,8.0,frequent customer
3,0,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,...,both,Regularly busy,Busiest days,Fewest orders,3,New customer,4.972414,Low spender,9.0,frequent customer
4,0,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,...,both,Least busy,Busiest days,Fewest orders,3,New customer,4.972414,Low spender,9.0,frequent customer


In [25]:
# user_id will be key column

In [26]:
# Determine data type in customer
df_customer['user_id'].dtype

dtype('int64')

In [27]:
# Determine data type in ords_prods_merge
ords_prods_merge['user_id'].dtype

dtype('int64')

In [34]:
df_merged = df_customer.merge(ords_prods_merge, on = 'user_id')

In [36]:
# Confirm merge
df_merged.head()

,user_id,First Name,Surname,Gender,State,Age,date_joined,n_dependants,fam_status,Income,...,_merge,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_price,spending_flag,med_freq,freq_flag
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Regularly busy,Busiest days,Fewest orders,8,New customer,7.988889,Low spender,19.0,Regular customer
1,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Regularly busy,Busiest days,Fewest orders,8,New customer,7.988889,Low spender,19.0,Regular customer
2,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Regularly busy,Busiest days,Fewest orders,8,New customer,7.988889,Low spender,19.0,Regular customer
3,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Regularly busy,Busiest days,Fewest orders,8,New customer,7.988889,Low spender,19.0,Regular customer
4,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665,...,both,Regularly busy,Busiest days,Fewest orders,8,New customer,7.988889,Low spender,19.0,Regular customer


In [37]:
# Create path
path = r'C:\Users\User\Documents\07-2022 Instacart Basket Analysis'

In [38]:
# Export data
df_merged.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customers_merged.pkl'))